# Clash of Clans CWL
## Create Dataframes for the Clan War Leagues after MidCWL check

* Your Name: Nolan Clark
* Name of API: Clash of Clans API
* Link to main page of API documentation: https://developer.clashofclans.com/#/documentation

In [1]:
# Imports go here
import requests
import pandas as pd
import numpy as np
import MyKeys
import json
import datetime
import time
import CWL_Functions as cf
from collections import Counter
from collections import defaultdict

In [2]:
# Start timer
start_all= time.time()
# Set pointer to current month
month=datetime.datetime.now().strftime('%B')

# Initiate requests session
MyToken = MyKeys.MyToken

headers = {'authorization': 'Bearer '+(MyToken), 'Accept': 'application/json'}
session = requests.Session()
session.headers.update(headers)

In [3]:
# open JSON file to update with current CWL
with open(f"PipelineRun{month}VERIFIED.json") as fp:
    dictObj = json.load(fp)

In [4]:
{key: (len(dictObj[key])) for key in  dictObj.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 0,
 'Silver League II': 0,
 'Silver League I': 0,
 'Gold League III': 588,
 'Gold League II': 1148,
 'Gold League I': 1820,
 'Crystal League III': 3164,
 'Crystal League II': 4172,
 'Crystal League I': 6916,
 'Master League III': 5096,
 'Master League II': 3248,
 'Master League I': 2520,
 'Champion League III': 2660,
 'Champion League II': 1400,
 'Champion League I': 700}

In [5]:
leagues = [ 'Gold League III',
            'Gold League II',
            'Gold League I',
            'Crystal League III',
            'Crystal League II',
            'Crystal League I',
            'Master League III',
            'Master League II',
            'Master League I',
            'Champion League III',
            'Champion League II',
            'Champion League I']

In [6]:
# Collect matchups for all rounds in all leagues - group number collect
start=time.time()


cwl_day_bucket = pd.DataFrame(data = None)
clan_count_bucket = Counter()
cwl_groups = defaultdict(list)
cwl_star_bucket = defaultdict(int)
cwl_damage_bucket = defaultdict(float)

#leagues =leagueBinsNotEmpty(dictObj)

# Clan Counter to avoid double grabbing -- optimized

for league in leagues:
    league_startTime = time.time()
    print(league)
    for warTag in dictObj[league]:
        war_id_transformed = warTag.strip('#')
        url=('https://api.clashofclans.com/v1/clanwarleagues/wars/%23'+(war_id_transformed))
        response = session.get(url,timeout=30)
        roundClanTags = [response.json()['clan'].get('tag'),response.json()['opponent'].get('tag')]
        clan_count_bucket.update(roundClanTags)
        
        if not ((clan_count_bucket.get(roundClanTags[0]) > 7) or (clan_count_bucket.get(roundClanTags[1]) > 7)):
            # group num collect
            cwl_groups[roundClanTags[0]].append(roundClanTags[1])
            cwl_groups[roundClanTags[1]].append(roundClanTags[0])
            # cwl star collect
            rs = cf.grab_cwl_stars(response)
            cwl_star_bucket[rs[0]]+=rs[1]
            cwl_star_bucket[rs[2]]+=rs[3]
            # cwl damage collect
            cwl_damage_bucket[rs[0]]+=rs[4]
            cwl_damage_bucket[rs[2]]+=rs[5]
            # individual performance collect
            df_day = cf.frame_attacks(response, league)
            cwl_day_bucket=cwl_day_bucket.append(df_day, ignore_index=True)
    league_endTime = time.time()
    print(league_endTime-league_startTime)
end=time.time()
print('All leauges complete: ',end-start)

Gold League III
66.30594682693481
Gold League II
95.70914006233215
Gold League I
219.34162783622742
Crystal League III
264.748202085495
Crystal League II
321.4285819530487
Crystal League I
668.7269690036774
Master League III
527.1435568332672
Master League II
369.0986258983612
Master League I
311.3942160606384
Champion League III
329.98680114746094
Champion League II
193.26857614517212
Champion League I
100.76033306121826
All leauges complete:  3467.9167442321777


In [7]:
# ADD key to group values before sorting and removing duplicates
for key, group in cwl_groups.items():
    group.append(key)
# list of sorted groups
group_sets =[sorted(item) for item in cwl_groups.values()]

In [8]:
# Unique list of groups in comprehension

cwl_unique_groups = [list(x) for x in set(tuple(x) for x in group_sets)]

unique_df=pd.DataFrame(cwl_unique_groups)
uni_df_T = unique_df.T

groups_for_merge = pd.melt(uni_df_T, var_name='group_num', value_name='clan_tag')

In [9]:
stars=pd.Series(cwl_star_bucket).to_frame(name = 'Tot_Clan_Stars')

In [10]:
clans_ds=pd.Series(cwl_damage_bucket).to_frame(name='Tot_Clan_Damage').join(stars)

In [11]:
stars_groups=groups_for_merge.set_index('clan_tag').join(clans_ds).reset_index()

In [12]:
# Rank the groups for final placement with total clan damage as tie breaker
stars_groups['group_rank'] = stars_groups.sort_values(by = 'Tot_Clan_Damage', 
                         ascending=False).groupby('group_num')['Tot_Clan_Stars'].rank(method = 'first',
                                                                                      ascending = False)

# Problem-Solution: including a total damage variable prevented 3.5 type rankings
# Validation check on group 201: Confirmed #9CGGRUUQ promo + #JUPJJV0G remained

In [13]:
def promo_results(x):
    if x == 1:
        return 'PROMOTED'
    elif x>=7:
        return 'DROPPED'
    else:
        return 'REMAINED'

In [14]:
stars_groups['result']=stars_groups['group_rank'].apply(promo_results)

In [15]:
# Write to file Clans measures
stars_groups.to_csv(f'{month}CWLgroupPerformanceDF2.csv', index=True)

In [16]:
cwl_df=cwl_day_bucket.merge(stars_groups[['clan_tag','group_rank','result']], on = 'clan_tag')

In [17]:
cwl_df.to_csv(f'{month}ModelDF2.csv', index=True)

In [18]:
end_all = time.time()
print(end_all - start_all)

3476.6054027080536


In [19]:
cwl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985320 entries, 0 to 985319
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   name          985320 non-null  object 
 1   tag           985320 non-null  object 
 2   clan          985320 non-null  object 
 3   clan_tag      985320 non-null  object 
 4   mapPosition   985320 non-null  int64  
 5   TH_lvl        985320 non-null  int64  
 6   A_stars       855527 non-null  float64
 7   A_Percent     855527 non-null  float64
 8   A_order       855527 non-null  float64
 9   A_duration    855527 non-null  float64
 10  D_stars       840011 non-null  float64
 11  D_Percent     840011 non-null  float64
 12  D_order       840011 non-null  float64
 13  D_duration    840011 non-null  float64
 14  league        985320 non-null  object 
 15  attack_trans  855527 non-null  float64
 16  def_trans     840011 non-null  float64
 17  star_diff     738869 non-null  float64
 18  grou

In [20]:
pd.unique(stars_groups.group_rank)

array([4., 5., 8., 6., 7., 3., 1., 2.])

In [21]:
cwl_df

,name,tag,clan,clan_tag,mapPosition,TH_lvl,A_stars,A_Percent,A_order,A_duration,D_stars,D_Percent,D_order,D_duration,league,attack_trans,def_trans,star_diff,group_rank,result
0,rin,#QJYLP0YGR,Sena Empiree,#2QGUG0UQP,9,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gold League III,NaN,NaN,NaN,6.0,REMAINED
1,Ardi septian,#QCLUQGUP9,Sena Empiree,#2QGUG0UQP,19,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gold League III,NaN,NaN,NaN,6.0,REMAINED
2,kuntex,#998UQQG0L,Sena Empiree,#2QGUG0UQP,4,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gold League III,NaN,NaN,NaN,6.0,REMAINED
3,PRABU SILIWANGI,#80VY2J0PQ,Sena Empiree,#2QGUG0UQP,3,15,3.0,100.0,6.0,148.0,NaN,NaN,NaN,NaN,Gold League III,300.0,NaN,NaN,6.0,REMAINED
4,•sanzZz•,#QV2L22VQP,Sena Empiree,#2QGUG0UQP,18,10,2.0,87.0,2.0,179.0,NaN,NaN,NaN,NaN,Gold League III,174.0,NaN,NaN,6.0,REMAINED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985315,呆萌小猪,#QVPV0VV2C,一杯浊酒笑风尘,#PJ0C0089,8,15,2.0,70.0,11.0,127.0,3.0,100.0,23.0,118.0,Champion League I,140.0,300.0,-1.0,6.0,REMAINED
985316,月永,#Q29Q8JYVU,一杯浊酒笑风尘,#PJ0C0089,3,15,2.0,65.0,20.0,83.0,3.0,100.0,19.0,143.0,Champion League I,130.0,300.0,-1.0,6.0,REMAINED
985317,宮脇丽莎,#QGPQYLULU,一杯浊酒笑风尘,#PJ0C0089,5,15,2.0,82.0,7.0,150.0,3.0,100.0,26.0,143.0,Champion League I,164.0,300.0,-1.0,6.0,REMAINED
985318,伝白,#QRL8LGGYV,一杯浊酒笑风尘,#PJ0C0089,9,15,1.0,15.0,4.0,32.0,3.0,100.0,28.0,169.0,Champion League I,15.0,300.0,-2.0,6.0,REMAINED
